In [9]:
%reload_ext autoreload
%autoreload 2
from torch_geometric.datasets import LRGBDataset
from torch_geometric.data import DataLoader


from models import ChebNet
from utils import compute_dirichlet_energy, train, test ,run_experiment

In [3]:
dataset = LRGBDataset(root="./data/LRGB", name="Peptides-func")
train_dataset = LRGBDataset(root="./data/LRGB", name="Peptides-func", split="train")
val_dataset = LRGBDataset(root="./data/LRGB", name="Peptides-func", split="test")

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
train_loader_for_energy = DataLoader(train_dataset, batch_size=10873, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

/opt/homebrew/anaconda3/envs/env/lib/python3.9/site-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [6]:
device = "cpu"

In [10]:
model = ChebNet(dataset.num_features, 64, dataset.num_classes, 5)
train_acc, val_acc, dirichlet_energy = run_experiment(train_loader, val_loader, train_loader_for_energy, model, device)

KeyboardInterrupt: 